<a href="https://colab.research.google.com/github/VenkateshTK/NOMA/blob/master/NOMA___working.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Non Orthogonal Multiple Access , channel capacity maximising.

In [0]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import pandas as pd 
import numpy as np
import torch.nn.functional as F
from torch.utils.data.dataset import Dataset

In [10]:
# dataset 
dataset_dir='/content/dataset.csv'

class CustomDataset(Dataset):
  def __init__(self,csv_path):
        self.to_tensor = transforms.ToTensor()
        self.coloum_names = ["h1","h2","h3","h4","p1","p2","p3","p4","pt,""cost"]
        df=pd.read_csv(csv_path)    
        self.h = np.asarray(df.iloc[:,0:4])
        self.power = np.asarray(df.iloc[:,4:8])
        self.power_total = np.asarray(df.iloc[:,8])
        self.data_len=len(df.index)

  def __len__(self):
        return self.data_len

  def __getitem__(self, index):
        data1=self.h[index]
        data2=self.power_total[index]
        input_data = np.concatenate([data1,data2],axis = None)
        #print(input_data)
        output_data = self.power[index]
        return torch.from_numpy(input_data),torch.from_numpy(output_data)

dataset=CustomDataset(dataset_dir)
print(dataset.__getitem__(0))

#print(dataframe.head())new_a = a[np.newaxis, :]
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset,[train_size, test_size])

(tensor([1.7978, 1.8785, 1.9903, 2.3837, 7.4438], dtype=torch.float64), tensor([1.9880, 0.4766, 0.9791, 4.0000], dtype=torch.float64))


In [11]:

input_size = 5
output_size= 4
hidden_size = 13
num_classes = 4
num_epochs = 50
batch_size = 1
learning_rate = 0.001


train_loader = torch.utils.data.DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,batch_size=10,shuffle=True)


# Fully connected neural network with one hidden layer
class Net(torch.nn.Module):
    def __init__(self, n_feature, n_hidden, n_output):
        super(Net, self).__init__()
        self.hidden_1 = torch.nn.Linear(n_feature, n_hidden)
        self.hidden_2 = torch.nn.Linear(n_hidden, n_hidden)    # hidden layer
        self.predict = torch.nn.Linear(n_hidden, n_output)   # output layer

    def forward(self, x):
        x = F.relu(self.hidden_1(x))
        x = F.relu(self.hidden_2(x))      # activation function for hidden layer
        x = self.predict(x)             # linear output
        return x


model = Net(input_size, hidden_size, num_classes)

print(model)

Net(
  (hidden_1): Linear(in_features=5, out_features=13, bias=True)
  (hidden_2): Linear(in_features=13, out_features=13, bias=True)
  (predict): Linear(in_features=13, out_features=4, bias=True)
)


In [0]:
# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)  

In [13]:

total_step = len(train_loader)
running_loss=0.0
loss_values=[]
for epoch in range(num_epochs):
    for i, (data,labels) in enumerate(train_loader):        
        # Forward pass
        labels=labels[:,np.newaxis]
        outputs = model(data.float())
        loss = criterion(outputs, labels.float())
        #print(outputs)
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #loss_values.append(loss.item())
        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, total_step, loss.item()))        
 
        if (i+1) % batch_size == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

torch.save(model.state_dict(), 'model.ckpt')

Epoch [1/50], Step [1/1092], Loss: 5.2941
Epoch [1/50], Step [1/1092], Loss: 5.2941
Epoch [1/50], Step [2/1092], Loss: 8.1409
Epoch [1/50], Step [2/1092], Loss: 8.1409
Epoch [1/50], Step [3/1092], Loss: 4.2665
Epoch [1/50], Step [3/1092], Loss: 4.2665
Epoch [1/50], Step [4/1092], Loss: 40.6315
Epoch [1/50], Step [4/1092], Loss: 40.6315
Epoch [1/50], Step [5/1092], Loss: 50.7234
Epoch [1/50], Step [5/1092], Loss: 50.7234
Epoch [1/50], Step [6/1092], Loss: 19.6632
Epoch [1/50], Step [6/1092], Loss: 19.6632
Epoch [1/50], Step [7/1092], Loss: 4.1613
Epoch [1/50], Step [7/1092], Loss: 4.1613
Epoch [1/50], Step [8/1092], Loss: 7.6580
Epoch [1/50], Step [8/1092], Loss: 7.6580
Epoch [1/50], Step [9/1092], Loss: 7.4500
Epoch [1/50], Step [9/1092], Loss: 7.4500
Epoch [1/50], Step [10/1092], Loss: 82.3542
Epoch [1/50], Step [10/1092], Loss: 82.3542
Epoch [1/50], Step [11/1092], Loss: 9.6278
Epoch [1/50], Step [11/1092], Loss: 9.6278
Epoch [1/50], Step [12/1092], Loss: 4.8089
Epoch [1/50], Step [1

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:431: UserWarning: Using a target size (torch.Size([1, 1, 4])) that is different to the input size (torch.Size([1, 4])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Streaming output truncated to the last 5000 lines.
Epoch [48/50], Step [777/1092], Loss: 0.0008
Epoch [48/50], Step [778/1092], Loss: 0.0005
Epoch [48/50], Step [778/1092], Loss: 0.0005
Epoch [48/50], Step [779/1092], Loss: 0.0006
Epoch [48/50], Step [779/1092], Loss: 0.0006
Epoch [48/50], Step [780/1092], Loss: 0.0033
Epoch [48/50], Step [780/1092], Loss: 0.0033
Epoch [48/50], Step [781/1092], Loss: 0.0010
Epoch [48/50], Step [781/1092], Loss: 0.0010
Epoch [48/50], Step [782/1092], Loss: 0.0003
Epoch [48/50], Step [782/1092], Loss: 0.0003
Epoch [48/50], Step [783/1092], Loss: 0.0022
Epoch [48/50], Step [783/1092], Loss: 0.0022
Epoch [48/50], Step [784/1092], Loss: 0.0018
Epoch [48/50], Step [784/1092], Loss: 0.0018
Epoch [48/50], Step [785/1092], Loss: 0.0009
Epoch [48/50], Step [785/1092], Loss: 0.0009
Epoch [48/50], Step [786/1092], Loss: 0.0006
Epoch [48/50], Step [786/1092], Loss: 0.0006
Epoch [48/50], Step [787/1092], Loss: 0.0000
Epoch [48/50], Step [787/1092], Loss: 0.0000
Epoc

In [14]:
model = Net(input_size, hidden_size, num_classes)
model.load_state_dict(torch.load('model.ckpt'))
model.eval()

# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    for (data,labels) in test_loader:
        #data=transforms.ToTensor(data)
        labels=labels[:,np.newaxis]
        outputs = model(data.float())
        loss = criterion(outputs,labels.float())



/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:431: UserWarning: Using a target size (torch.Size([10, 1, 4])) that is different to the input size (torch.Size([10, 4])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:431: UserWarning: Using a target size (torch.Size([4, 1, 4])) that is different to the input size (torch.Size([4, 4])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [15]:
print(outputs)
print(labels)
print(data[0])        

        


tensor([[ 5.0021,  2.2253,  2.4921, 10.2627],
        [ 2.8887,  1.1078,  1.3946,  5.8547],
        [ 4.4610,  1.9151,  2.2154,  9.2168],
        [ 2.5633,  0.9853,  1.2375,  5.2444]])
tensor([[[ 5.0119,  2.2344,  2.4704, 10.2334]],

        [[ 2.8900,  1.0875,  1.4044,  5.8381]],

        [[ 4.4627,  1.9204,  2.2113,  9.1856]],

        [[ 2.5725,  0.9719,  1.2166,  5.2390]]], dtype=torch.float64)
tensor([ 1.9352,  3.2562,  4.2370,  4.5350, 19.9500], dtype=torch.float64)
